In [1]:
include("src//tokens.jl")

using ..HllSets
using ..Tokens
using ..Util

using Redis
using EasyConfig
using UUIDs
using Random
using JSON3

conn = Redis.RedisConnection()

Tokens.token_idx(conn, "b:tokens", "b:token")

"OK"

In [2]:
name = "masha"
token_id = Tokens.create_token_id(name, "b")
node_sha1 = "d6a088fc288h866b0e5e46b9375afe402dfe6b62"

_hash = Redis.hgetall(conn, token_id)
if _hash == nothing || isempty(_hash)
    token = Tokens.create_token(name, node_sha1)
else
    token = Tokens.Token(_hash)
end

Tokens.set_token(conn, token, node_sha1, "b")

true